In [ ]:
# %pip install numpy
# %pip install scipy
# %pip install scikit-learn
# %pip install matplotlib
import numpy as np
from scipy.optimize import fsolve
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from scipy import integrate
from scipy.optimize import minimize, NonlinearConstraint

In [ ]:
mu = 0.5 # Static Tire Friction Coefficient
g = 9.8 # m/s^2 acceleration due to gravity
# FAW = 291.58 # kg
# RAW = 380.38 # kg
Cf = 300 # Cornering Stiffness of front tires in N/deg
Cr = 450 # Cornering Stiffness of rear tires in N/deg

In [ ]:
class Vehicle:
    """The Vehicle class encapsulates a variety of methods and properties related to vehicle dynamics, steering, wheel loads, and geometric calculations.
     It leverages numerical methods, regressions, and physical equations to model and analyze vehicle behavior under different conditions.
     The class is designed to provide comprehensive insights into the performance and characteristics of a vehicle's steering and suspension systems.
    """
    # --- Constuctor for Inputs ---
    def __init__(self, r_A: np.array,
             r_B: np.array,
             r_C: np.array,
             r_O: np.array,
             r_K: np.array,
             slr: float,
             initial_camber: float,
             tw: float,
             wb: float,
             GVW: float,
             b: float,
             wheel_rate_f: float,
             wheel_rate_r: float,
             tire_stiffness_f: float,
             tire_stiffness_r: float,
             pinion: float,
             tirep: float,
             dila: float,
             r_La: np.array,
             r_Lb: np.array,
             r_strut: np.array = np.array([0, 0, 0]),
             r_Ua: np.array = np.array([0, 0, 0]),
             r_Ub: np.array = np.array([0, 0, 0])):
        """
        Initializes parameters for Steering Design.

        Args:
        r_A (ndarray): Upper Ball Joint (BJ), Top Strut Mount for MacPherson.
        r_B (ndarray): OBJ of Tie Rod to make AB as Steering Arm.
        r_C (ndarray): IBJ of Tie Rod (Rack Vector Definition point) to make BC as tie rod.
        r_O (ndarray): Wheel Center.
        r_K (ndarray): Lower Ball Joint (BJ) to make KA as Kingpin Axis.
        slr (float): Static Load Radius in mm.
        initial_camber (float): Initial Camber angle in degrees.
        tw (float): Trackwidth in mm.
        wb (float): Wheelbase in mm.
        GVW (float): Gross Vehicle Weight in kg.
        b (float): CG distance from rear axle.
        wheel_rate_f (float): Wheel Rate for Front Wheels in kg/mm.
        wheel_rate_r (float): Wheel Rate for Rear Wheels in kg/mm.
        tire_stiffness_f (float): Tire Stiffness for Front Wheels in kg/mm.
        tire_stiffness_r (float): Tire Stiffness for Rear Wheels in kg/mm.
        pinion (float): Pinion Radius in mm.
        tirep (float): Tire Pressure in psi.
        dila (float): Designed inner lock angle (road steer angle).
        r_La (ndarray): Lower Control Arm Pivot; default: np.array([0,0,0]).
        r_Lb (ndarray): Lower Control Arm Pivot; default: np.array([0,0,0]).
        r_strut (ndarray): Strut Lower Mount for MacPherson; default: np.array([0,0,0]).
        r_Ua (ndarray): Upper Control Arm Pivot; default: np.array([0,0,0]).
        r_Ub (ndarray): Upper Control Arm Pivot; default: np.array([0,0,0]).

        Notes:
        - Sets instance variables based on input parameters.
        - Computes various projections and helper variables for steering geometry.
        """
        # Assigning instance variables based on input parameters
        self.r_A = r_A
        self.r_B = r_B
        self.r_C = r_C
        self.r_O = r_O
        self.r_K = r_K
        self.slr = slr
        self.initial_camber = initial_camber
        self.tw = tw
        self.wb = wb
        self.GVW = GVW
        self.b = b
        self.a = wb - b  # Distance from rear axle to CG in mm
        self.Kf = wheel_rate_f * tire_stiffness_f / (wheel_rate_f + tire_stiffness_f)  # Front wheel rate
        self.Kr = wheel_rate_r * tire_stiffness_r / (wheel_rate_r + tire_stiffness_r)  # Rear wheel rate
        self.pinion = pinion
        self.tirep = tirep
        self.dila = dila
        self.step = 0.1
        self.maxdecimal = int(-np.log10(self.step))
        self.conversionstep = int(10**self.maxdecimal)
        # Calculating additional points based on initial parameters
        self.r_D = np.array([self.r_C[0], 0.00, self.r_C[2]])
        self.r_T = np.array([self.r_O[0], self.r_O[1] - slr * np.sin(np.radians(self.initial_camber)),
                            self.r_O[2] - self.slr])
        self.r_O[2] = self.r_O[2] - self.slr + slr * np.cos(np.radians(self.initial_camber))
        self.r_W = self.r_T + np.array([-1, 0, 0])

        # Calculating Kingpin Axis and related angles
        self.KPA = (r_A - r_K) / Vehicle.magnitude(r_A - r_K)  # Kingpin Axis
        self.currKPA = (r_A - r_K) / Vehicle.magnitude(r_A - r_K)  # Current Kingpin Axis

        # Assigning additional instance variables
        self.r_La = r_La
        self.r_Lb = r_Lb
        self.r_strut = r_strut
        self.r_Ua = r_Ua
        self.r_Ub = r_Ub

        # Initializing arrays for calculation purposes
        self.mindp = 50
        self.maxdp = 50
        self.dpK = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpO = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpT = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpW = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpA = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpB = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpnewB = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpC = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpdz = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1)))
        self.dpfvsa = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.dpsvsa = np.zeros((int(self.mindp / self.step + self.maxdp / self.step + 1), 3))
        self.zeropos = int(self.mindp / self.step)
        self.dpK[self.zeropos] = self.r_K
        self.dpO[self.zeropos] = self.r_O
        self.dpT[self.zeropos] = self.r_T
        self.dpW[self.zeropos] = self.r_W
        self.dpA[self.zeropos] = self.r_A
        self.dpB[self.zeropos] = self.r_B
        self.dpnewB[self.zeropos] = self.r_B
        self.dpC[self.zeropos] = self.r_C
        self.dpdz[self.zeropos] = 0
        self.dpfvsa[self.zeropos] = self.fvsa_ic(0)
        self.dpsvsa[self.zeropos] = self.svsa_ic(0)

        # Calculating caster and kingpin inclination angles
        h1 = (self.KPA - np.dot(self.KPA, np.array([0, 1, 0])) * np.array([0, 1, 0])) / Vehicle.magnitude(
            self.KPA - np.dot(self.KPA, np.array([0, 1, 0])) * np.array([0, 1, 0]))
        h2 = (self.KPA - np.dot(self.KPA, np.array([1, 0, 0])) * np.array([1, 0, 0])) / Vehicle.magnitude(
            self.KPA - np.dot(self.KPA, np.array([1, 0, 0])) * np.array([1, 0, 0]))
        self.caster = np.degrees(np.arccos(np.dot(h1, np.array([0, 0, 1]))))
        self.kpi = np.degrees(np.arccos(np.dot(h2, np.array([0, 0, 1]))))

        # Calculating projection points
        t = (self.r_T[2] - self.r_K[2]) / self.KPA[2]
        self.r_I = self.r_K + t * self.KPA
        self.r_Aprime = Vehicle.projection(self.r_A, self.KPA, self.r_B)
        self.r_Iprime = Vehicle.projection(self.r_A, self.KPA, self.r_T)
        self.r_Iwprime = Vehicle.projection(self.r_A, self.KPA, self.r_W)
        self.r_Ioprime = Vehicle.projection(self.r_A, self.KPA, self.r_O)

        # Initializing additional helper variables and methods
        self.model = self.regression_model()
        self.rack_stroke = self.rack_vs_road_steer(self.dila)
        self.curr_KPA_angle = 0
        self.curr_KPA_angle_for_fvsa = 0
        self.curr_KPA_angle_for_svsa = 0
        self.curr_KPA_angle_for_K = 0
        self.curr_KPA_angle_for_A = 0
    # --- Calculation of instantaneous axis for suspension travel ---
    def fvsa_equations(self, values):
        """
        Calculates the equations for Front View Swing Arm (FVSA) optimization.

        Computes the difference between two vectors based on vehicle geometry and steering parameters
        to find optimal values of `la` and `mu`. Depending on whether `r_strut` is defined, calculates
        equations for suspension parameters affecting FVSA optimization.

        Args:
        values (list or tuple): Contains two float values representing:
            - `la`: Parameter affecting the vector calculation based on current_A and current_K.
            - `mu`: Parameter affecting the vector calculation based on current_K and average of r_La and r_Lb.

        Returns:
        list: A list containing two equations (`eq1` and `eq2`) representing the difference between `l2` and `l1`.
            - `eq1`: Difference in the y-component between `l2` and `l1`.
            - `eq2`: Difference in the z-component between `l2` and `l1`.

        Notes:
        - If `r_strut` is not defined (equal to [0, 0, 0]), calculates `a2` based on average of r_Ua and r_Ub.
        - If `r_strut` is defined, calculates `a2` based on current_A and cross product of r_strut-a1 and [1, 0, 0].
        - `current_A`, `current_K`, and `current_O` are calculated using `self.curr_A`, `self.curr_K`, and `self.curr_O`
        methods respectively, with `self.curr_KPA_angle_for_fvsa` as input.
        """
        if(self.r_strut[0] == 0):
            la = values[0]
            mu = values[1]
            current_A = self.curr_A(self.curr_KPA_angle_for_fvsa)
            current_K = self.curr_K(self.curr_KPA_angle_for_fvsa)
            a1 = current_A
            a2 = (self.r_Ua+self.r_Ub)/2
            b1 = current_K
            b2 = (self.r_La+self.r_Lb)/2
            a2 += 1e-9
            b2 += 1e-9
            l1 = a1 + la*(a1-a2)
            l2 = b1 + mu*(b1-b2)
            eq1 = (l2-l1)[1]
            eq2 = (l2-l1)[2]
        else:
            la = values[0]
            mu = values[1]
            current_A = self.curr_A(self.curr_KPA_angle_for_fvsa)
            current_K = self.curr_K(self.curr_KPA_angle_for_fvsa)
            current_O = self.curr_O(self.curr_KPA_angle_for_fvsa)
            a1 = current_A
            a2 = a1+np.cross(self.r_strut-a1, np.array([1,0,0]))
            b1 = current_K
            b2 = (self.r_La+self.r_Lb)/2
            a2 += 1e-9
            b2 += 1e-9
            l1 = a1 + la*(a1-a2)
            l2 = b1 + mu*(b1-b2)
            eq1 = (l2-l1)[1]
            eq2 = (l2-l1)[2]
        return [eq1,eq2]
    def fvsa_ic(self, curr_KPA_angle):
        """
        Computes the Instantaneous Centers (IC) for the Front View Swing Arm (FVSA) suspension.

        This method calculates the IC based on the current KPA angle and the geometry of the suspension.

        Args:
        curr_KPA_angle (float): Current Kingpin Axis (KPA) angle in degrees.

        Returns:
        ndarray: Coordinates of the Instantaneous Center (IC) in 3D space.

        Notes:
        - Uses numerical root-finding (fsolve) to solve the FVSA equations for la and mu.
        - Handles different configurations based on the presence of a strut.
        """
        self.curr_KPA_angle_for_fvsa = curr_KPA_angle
        position_to_add = self.zeropos + int(np.round(curr_KPA_angle, self.maxdecimal) * self.conversionstep)
        if(self.dpfvsa[position_to_add][0]>self.dpfvsa[self.zeropos][0]/10):
            return self.dpfvsa[position_to_add]
        if self.r_strut[0] == 0:
            # No strut present
            la, mu = fsolve(self.fvsa_equations, [0.01, 0.01])
            current_A = self.curr_A(curr_KPA_angle)
            current_K = self.curr_K(curr_KPA_angle)
            a1 = current_A
            a2 = (self.r_Ua + self.r_Ub) / 2
            b1 = current_K
            b2 = (self.r_La + self.r_Lb) / 2
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
        else:
            # Strut present
            la, mu = fsolve(self.fvsa_equations, [0.01, 0.01])
            current_A = self.curr_A(curr_KPA_angle)
            current_K = self.curr_K(curr_KPA_angle)
            current_O = self.curr_O(curr_KPA_angle)
            a1 = current_A
            a2 = a1 + np.cross(self.r_strut - a1, np.array([1, 0, 0]))
            b1 = current_K
            b2 = (self.r_La + self.r_Lb) / 2
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
        self.dpfvsa[position_to_add] = (l1 + l2) / 2
        return self.dpfvsa[position_to_add]
    def svsa_equations(self, values):
        """
        Calculates the Side View Swing Arm (SVSA) suspension equations for finding la and mu.

        This method computes the equations based on the current configuration of the suspension.
        For configurations without a strut, it uses the upper (Ua, Ub) and lower (La, Lb) control arm pivot points.
        For configurations with a strut, it adjusts the calculation based on the strut position relative to the upper pivot.

        Args:
        values (list): List containing la and mu values to solve the equations.

        Returns:
        list: Equations [eq1, eq2] representing the difference between computed lengths l2 and l1 along x and z axes.

        Notes:
        - Uses current KPA angle for calculating current_A.
        - Handles different suspension configurations based on the presence of a strut (r_strut).
        """
        if self.r_strut[0] == 0:
            # No strut present
            la = values[0]
            mu = values[1]
            current_A = self.curr_A(self.curr_KPA_angle_for_svsa)
            a1 = self.r_Ua
            a2 = self.r_Ub
            b1 = self.r_La
            b2 = self.r_Lb
            a2 += 1e-9
            b2 += 1e-9
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
            eq1 = (l2 - l1)[0]
            eq2 = (l2 - l1)[2]
        else:
            # Strut present
            la = values[0]
            mu = values[1]
            current_A = self.curr_A(self.curr_KPA_angle_for_svsa)
            a1 = current_A
            a2 = a1 + np.cross(self.r_strut - a1, np.array([0, 1, 0]))
            b1 = self.r_La
            b2 = self.r_Lb
            a2 += 1e-9
            b2 += 1e-9
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
            eq1 = (l2 - l1)[0]
            eq2 = (l2 - l1)[2]
        
        return [eq1, eq2]
    def svsa_ic(self, curr_KPA_angle):
        """
        Computes the Instantaneous Centers (IC) for the Side View Swing Arm (SVSA) suspension.

        This method calculates the IC based on the current configuration of the SVSA suspension.
        If no strut is present, it uses the upper (Ua, Ub) and lower (La, Lb) control arm pivot points.
        If a strut is present, it adjusts the calculation based on the strut position relative to the upper pivot.

        Args:
        curr_KPA_angle (float): Current Kingpin Axis (KPA) angle in radians.

        Returns:
        ndarray: Coordinates of the IC (Instantaneous Center) calculated as the midpoint of lengths l1 and l2.

        Notes:
        - Uses fsolve to solve the svsa_equations for la and mu.
        - Handles different suspension configurations based on the presence of a strut (r_strut).
        """
        self.curr_KPA_angle_for_svsa = curr_KPA_angle
        position_to_add = self.zeropos + int(np.round(curr_KPA_angle, self.maxdecimal) * self.conversionstep)
        if(self.dpsvsa[position_to_add][0]>self.dpsvsa[self.zeropos][0]/10):
            return self.dpsvsa[position_to_add]
        if self.r_strut[0] == 0:
            # No strut present
            [la, mu] = fsolve(self.svsa_equations, [0.01, 0.01])
            current_A = self.curr_A(curr_KPA_angle)
            a1 = self.r_Ua
            a2 = self.r_Ub
            b1 = self.r_La
            b2 = self.r_Lb
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
        else:
            # Strut present
            [la, mu] = fsolve(self.svsa_equations, [0.01, 0.01])
            current_A = self.curr_A(curr_KPA_angle)
            a1 = current_A
            a2 = a1 + np.cross(self.r_strut - a1, np.array([0, 1, 0]))
            b1 = self.r_La
            b2 = self.r_Lb
            l1 = a1 + la * (a1 - a2)
            l2 = b1 + mu * (b1 - b2)
        self.dpsvsa[position_to_add] = (l1 + l2) / 2
        return self.dpsvsa[position_to_add]
    def solveK(self, inputval):
        """
        Solves for the position of point K along the suspension axis.

        This method calculates the position of point K based on the input value `t` and the current configuration
        of the suspension. It uses the current orientation and positions of points O, K, and the instantaneous
        centers (ICs) for the FVSA and SVSA suspensions.

        Args:
        inputval (list): Input value `t` as a list where `t = [t_val]`.

        Returns:
        list: A list containing the equation `eq1`, representing the error between the current and previous delta_z values
            and the difference between `tempK` and the stored position of K.

        Notes:
        - Uses the current KPA angle (`curr_KPA_angle_for_K`) to calculate positions and orientations.
        - Adjusts positions based on the sign of `curr_KPA_angle_for_K`.
        - Utilizes rotational transformations and delta_z calculations for accuracy.
        """
        t = inputval[0]
        tempO = self.curr_O(self.curr_KPA_angle_for_K)
        position_to_add = self.zeropos + int(np.round(self.curr_KPA_angle_for_K, self.maxdecimal) * self.conversionstep)
        tempK = Vehicle.rotation(self.dpK[position_to_add - int(np.sign(self.curr_KPA_angle_for_K))].tolist(),
                                self.fvsa_ic(self.curr_KPA_angle_for_K - np.sign(self.curr_KPA_angle_for_K) * self.step).tolist(),
                                self.svsa_ic(self.curr_KPA_angle_for_K - np.sign(self.curr_KPA_angle_for_K) * self.step).tolist(), t)
        eq1 = self.delta_z(self.curr_KPA_angle_for_K) - self.delta_z(self.curr_KPA_angle_for_K - self.step * np.sign(self.curr_KPA_angle_for_K)) + (tempK - self.dpK[position_to_add - int(np.sign(self.curr_KPA_angle_for_K))])[2]
        return [eq1]
    def curr_K(self, curr_KPA_angle):
        """
        Computes the position of point K based on the current KPA angle.

        This method calculates the position of point K along the suspension axis based on the provided current
        Kingpin Axis (KPA) angle. If the angle is zero, it returns the initial position of point K. Otherwise,
        it adjusts the position using the stored positions and angles of various suspension components and ICs.

        Args:
        curr_KPA_angle (float): Current Kingpin Axis (KPA) angle in degrees.

        Returns:
        ndarray: The current position of point K in 3D space.

        Notes:
        - If `curr_KPA_angle` is zero, returns `self.r_K`.
        - Uses the stored positions (`dpK`) and ICs (`fvsa_ic` and `svsa_ic`) to compute the current position of K.
        - Adjusts positions based on the sign of `curr_KPA_angle` for accuracy.
        """
        if curr_KPA_angle == 0:
            return self.r_K

        position_to_add = self.zeropos + int(np.round(curr_KPA_angle, self.maxdecimal) * self.conversionstep)

        # Adjust position based on stored data and IC calculations
        if self.dpK[position_to_add][0] < self.r_K[0] / 10:
            self.curr_KPA_angle_for_K = curr_KPA_angle
            [t] = fsolve(self.solveK, [0.01])
            self.dpK[position_to_add] = Vehicle.rotation(self.dpK[position_to_add - int(np.sign(curr_KPA_angle))].tolist(),
                                                        self.fvsa_ic(curr_KPA_angle - np.sign(curr_KPA_angle) * self.step).tolist(),
                                                        self.svsa_ic(curr_KPA_angle - np.sign(curr_KPA_angle) * self.step).tolist(), t)

        return self.dpK[position_to_add]
    def curr_KPA(self, curr_KPA_angle):
        """
        Computes the current Kingpin Axis (KPA) based on the given KPA angle.

        This method calculates the current Kingpin Axis (KPA) by determining the vector difference between
        the current position of point A (upper ball joint) and point K (lower ball joint) at the given
        Kingpin Axis angle. It normalizes this vector to obtain the direction of the KPA.

        Args:
        curr_KPA_angle (float): The current Kingpin Axis (KPA) angle in degrees.

        Returns:
        ndarray: The normalized direction vector of the current Kingpin Axis (KPA).

        Notes:
        - The method updates the instance variable `self.currKPA` to the computed KPA direction.
        - Uses the methods `curr_A` and `curr_K` to obtain the current positions of points A and K.
        """
        t = self.curr_A(curr_KPA_angle) - self.curr_K(curr_KPA_angle)
        self.currKPA = t / Vehicle.magnitude(t)
        return t / Vehicle.magnitude(t)
    def solveA(self, inputval):
        """
        Solves for the rotation parameter `t` to determine the current position of point A (upper ball joint) at the specified Kingpin Axis (KPA) angle.

        This method computes the rotation parameter `t` needed to align the position of point A (upper ball joint)
        using the Front View Swing Arm (FVSA) and Side View Swing Arm (SVSA) intersection points.
        The method solves for `t` by ensuring the vertical displacement consistency between successive KPA angles.

        Args:
        inputval (list): A list containing the initial guess for the rotation parameter `t`.

        Returns:
        list: A list containing the equation residual that needs to be solved to find `t`.

        Notes:
        - The method uses `curr_O` to obtain the current position of point O (wheel center).
        - The position to add is determined based on the current KPA angle.
        - The temporary position of point A (`tempA`) is computed using the `rotation` method.
        - The vertical displacement (`delta_z`) consistency is checked between successive KPA angles.
        """
        t = inputval[0]
        tempO = self.curr_O(self.curr_KPA_angle_for_A)
        position_to_add = self.zeropos + int(np.round(self.curr_KPA_angle_for_A, self.maxdecimal) * self.conversionstep)
        tempA = Vehicle.rotation(
            self.dpA[position_to_add - int(np.sign(self.curr_KPA_angle_for_A))].tolist(),
            self.fvsa_ic(self.curr_KPA_angle_for_A - np.sign(self.curr_KPA_angle_for_A) * self.step).tolist(),
            self.svsa_ic(self.curr_KPA_angle_for_A - np.sign(self.curr_KPA_angle_for_A) * self.step).tolist(),
            t
        )
        eq1 = self.delta_z(self.curr_KPA_angle_for_A) - self.delta_z(self.curr_KPA_angle_for_A - self.step * np.sign(self.curr_KPA_angle_for_A)) + (tempA - self.dpA[position_to_add - int(np.sign(self.curr_KPA_angle_for_A))])[2]
        return [eq1]
    def curr_A(self, curr_KPA_angle):
        """
        Determines the current position of point A (upper ball joint) at the specified Kingpin Axis (KPA) angle.

        This method calculates the position of point A by solving for the rotation parameter `t` if the position
        has not been previously computed for the given KPA angle. It updates the position in the dpA array.

        Args:
        curr_KPA_angle (float): The current Kingpin Axis (KPA) angle in degrees.

        Returns:
        ndarray: The position of point A at the specified KPA angle.

        Notes:
        - If the `curr_KPA_angle` is zero, it returns the initial position `r_A`.
        - The method checks if the position for the given angle has already been computed.
        - If not, it solves for the rotation parameter `t` using the `solveA` method.
        - The position is then updated in the `dpA` array using the `rotation` method.
        """
        if curr_KPA_angle == 0:    
            return self.r_A
        position_to_add = self.zeropos + int(np.round(curr_KPA_angle, self.maxdecimal) * self.conversionstep)
        if self.dpA[position_to_add][0] < self.r_A[0] / 10:
            self.curr_KPA_angle_for_A = curr_KPA_angle
            [t] = fsolve(self.solveA, [0.01])
            self.dpA[position_to_add] = Vehicle.rotation(
                self.dpA[position_to_add - int(np.sign(curr_KPA_angle))].tolist(),
                self.fvsa_ic(curr_KPA_angle - np.sign(curr_KPA_angle) * self.step).tolist(),
                self.svsa_ic(curr_KPA_angle - np.sign(curr_KPA_angle) * self.step).tolist(),
                t
            )
        return self.dpA[position_to_add]

# --- Projection of point (x,y,z) on the plane a*x + b*y + c*z = 1 --- 
    def project_points(x, y, z, a, b, c):
        """
        Projects the points with coordinates x, y, z onto the plane
        defined by a*x + b*y + c*z = 1
        """
        vector_norm = a*a + b*b + c*c
        normal_vector = np.array([a, b, c]) / np.sqrt(vector_norm)
        point_in_plane = np.array([a, b, c]) / vector_norm

        points = np.column_stack((x, y, z))
        points_from_point_in_plane = points - point_in_plane
        proj_onto_normal_vector = np.dot(points_from_point_in_plane,
                                        normal_vector)
        proj_onto_plane = (points_from_point_in_plane -
                        proj_onto_normal_vector[:, None]*normal_vector)

        return point_in_plane + proj_onto_plane
    # --- Magnitude of a vector ---
    def magnitude(vector):
        return np.sqrt(sum(pow(element, 2) for element in vector))
    # --- Matrix Multiplication ---
    @staticmethod
    def safe_normalize(U):
        norm = np.linalg.norm(U)
        if norm == 0:
            return np.zeros_like(U)
        return U / norm

    @staticmethod
    def matrix_multiply(*matrices):
        result = matrices[0]
        for matrix in matrices[1:]:
            result = np.dot(result, matrix)
        return result
    # --- Rotaion of point p at an angle t about the axis defined by points x1,x2 ---
    def rotation(p, x1, x2, t):
        theta = np.radians(t)
        p = np.array([[pp] for pp in p] + [[1]])
        x1, y1, z1 = x1
        x2, y2, z2 = x2

        # Define the unit vector U along the axis of rotation
        U = np.array([x2 - x1, y2 - y1, z2 - z1])
        U = Vehicle.safe_normalize(U)
        a, b, c = U

        d = np.sqrt(b**2 + c**2)
        if d == 0:
            d = 1e-9  # Handle case where b and c are both zero to avoid division by zero

        # Translation matrices
        T = np.array([
            [1, 0, 0, -x1],
            [0, 1, 0, -y1],
            [0, 0, 1, -z1],
            [0, 0, 0, 1]
        ])
        T_inv = np.array([
            [1, 0, 0, x1],
            [0, 1, 0, y1],
            [0, 0, 1, z1],
            [0, 0, 0, 1]
        ])

        # Rotation matrices around x, y, and z axes
        R_x = np.array([
            [1, 0, 0, 0],
            [0, c / d, -b / d, 0],
            [0, b / d, c / d, 0],
            [0, 0, 0, 1]
        ])
        R_x_inv = np.array([
            [1, 0, 0, 0],
            [0, c / d, b / d, 0],
            [0, -b / d, c / d, 0],
            [0, 0, 0, 1]
        ])

        R_y = np.array([
            [d, 0, -a, 0],
            [0, 1, 0, 0],
            [a, 0, d, 0],
            [0, 0, 0, 1]
        ])
        R_y_inv = np.array([
            [d, 0, a, 0],
            [0, 1, 0, 0],
            [-a, 0, d, 0],
            [0, 0, 0, 1]
        ])

        # Rotation matrix around z-axis
        ct = np.cos(theta)
        st = np.sin(theta)
        R_z = np.array([
            [ct, st, 0, 0],
            [-st, ct, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

        # Composite transformation
        p_transformed = Vehicle.matrix_multiply(T_inv, R_x_inv, R_y_inv, R_z, R_y, R_x, T, p)

        return p_transformed[:3, 0]
    # --- Projection of a point given normal and point on plane ---
    def projection(point, normal, point_on_plane):
        """
        Projects the vector point on the plane with normal vector and point_on_plane vector
        """
        x=point[0]
        y=point[1]
        z=point[2]
        a=normal[0]/np.dot(normal,point_on_plane)
        b=normal[1]/np.dot(normal,point_on_plane)
        c=normal[2]/np.dot(normal,point_on_plane)
        return Vehicle.project_points(x,y,z,a,b,c)[0]
    # --- Local X and Y axes for the given centre, point and normal ---
    # --- Current Coordinates of points B,C,W,T and wheel travel in Z ---
    def old_B(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return self.r_B
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpB[position_to_add][0]<self.r_B[0]/10):
            self.dpB[position_to_add] = Vehicle.rotation(self.dpnewB[position_to_add-int(np.sign(curr_KPA_angle))].tolist(), self.curr_A(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(),self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(), np.sign(curr_KPA_angle)*self.step)          
        return self.dpB[position_to_add]
    def curr_B(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return self.r_B
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpnewB[position_to_add][0]<self.r_B[0]/10):
            self.dpnewB[position_to_add] = self.old_B(curr_KPA_angle) + self.curr_K(curr_KPA_angle) - self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle))
        return self.dpnewB[position_to_add]
    def curr_C(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return self.r_C
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpC[position_to_add][0]<self.r_C[0]/10):
            length = Vehicle.magnitude(self.r_C-self.r_B)
            temp = self.curr_B(curr_KPA_angle)
            self.dpC[position_to_add] = np.array([self.r_C[0],temp[1]-np.sqrt(length**2-(self.r_C[0]-temp[0])**2-(self.r_C[2]-temp[2])**2), self.r_C[2]])
        return self.dpC[position_to_add]
    def curr_W(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return self.r_W
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpW[position_to_add][0]<self.r_W[0]/10):
            temp = Vehicle.rotation(self.dpW[position_to_add-int(np.sign(curr_KPA_angle))].tolist(), self.curr_A(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(),self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(), np.sign(curr_KPA_angle)*self.step)
            temp[2] = self.r_W[2]
            self.dpW[position_to_add] = temp
        return self.dpW[position_to_add]
    def curr_T(self, curr_KPA_angle):
        
        if curr_KPA_angle==0:
            return self.r_T
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpT[position_to_add][0]<self.r_T[0]/10):
            temp = Vehicle.rotation(self.dpT[position_to_add-int(np.sign(curr_KPA_angle))].tolist(), self.curr_A(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(),self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(), np.sign(curr_KPA_angle)*self.step)
            temp[2] = self.r_T[2]
            self.dpT[position_to_add] = temp
        return self.dpT[position_to_add]
    def delta_z(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return 0
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpdz[position_to_add]==0):
            self.dpdz[position_to_add] = self.dpdz[position_to_add-int(np.sign(curr_KPA_angle))]+Vehicle.rotation(self.dpT[position_to_add-int(np.sign(curr_KPA_angle))].tolist(), self.curr_A(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(),self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(), np.sign(curr_KPA_angle)*self.step)[2] - self.r_T[2]
        return self.dpdz[position_to_add]
    def curr_O(self, curr_KPA_angle):
        if curr_KPA_angle==0:
            return self.r_O
        position_to_add = self.zeropos+int(np.round(curr_KPA_angle,self.maxdecimal)*self.conversionstep)
        if(self.dpO[position_to_add][0]<self.r_O[0]/10):
            temp = Vehicle.rotation(self.dpO[position_to_add-int(np.sign(curr_KPA_angle))].tolist(), self.curr_A(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(),self.curr_K(curr_KPA_angle-self.step*np.sign(curr_KPA_angle)).tolist(), np.sign(curr_KPA_angle)*self.step)
            temp[2] = temp[2] - self.delta_z(curr_KPA_angle) + self.delta_z(curr_KPA_angle-self.step*np.sign(curr_KPA_angle))
            self.dpO[position_to_add] = temp
        return self.dpO[position_to_add]
    # --- Current Tangent Motion of the Tire Contact Patch, returns the direction ---
    def curr_tangent(self, point):
        temp = Vehicle.projection(self.r_A,self.currKPA,point) - point
        product = np.cross(temp, self.currKPA)
        ans = np.array([product[0], product[1], 0])
        ans = ans/Vehicle.magnitude(ans)
        return ans
    # --- Rack Displacement, Steering Arm and Tie Rod ---
    def rack_displacement(self, curr_KPA_angle):
        return self.curr_C(curr_KPA_angle)[1]-self.r_C[1]
    def steering_arm(self, curr_KPA_angle):
        temp = self.curr_B(curr_KPA_angle)
        self.currKPA = (self.curr_A(curr_KPA_angle)-self.curr_K(curr_KPA_angle))/Vehicle.magnitude(self.r_A-self.r_K)
        return temp-Vehicle.projection(self.r_A, self.currKPA, temp)
    def tierod(self, curr_KPA_angle):
        return self.curr_C(curr_KPA_angle)-self.curr_B(curr_KPA_angle)
    # --- Inclination and Heading ---
    def wheel_inclination(self, curr_KPA_angle):
        return self.curr_O(curr_KPA_angle)-self.curr_T(curr_KPA_angle)
    def wheel_heading(self, curr_KPA_angle):
        
        return self.curr_W(curr_KPA_angle)-self.curr_T(curr_KPA_angle)
    # --- Caster Trail, Scrub Radius and Spindle Length ---
    def trails(self, curr_KPA_angle):
        return self.curr_T(curr_KPA_angle)-self.r_I
    def caster_trail(self, curr_KPA_angle):
        head = self.wheel_heading(curr_KPA_angle)
        mag = Vehicle.magnitude(head)
        return -np.dot(self.trails(curr_KPA_angle),head/mag)
    def scrub_radius(self, curr_KPA_angle):
        head = self.wheel_heading(curr_KPA_angle)
        mag  = Vehicle.magnitude(head)
        return np.sign(np.dot(self.trails(curr_KPA_angle),
                                np.cross(head,np.array([0,0,1]))))*Vehicle.magnitude(self.trails(curr_KPA_angle) + 
                                                                                     self.caster_trail(curr_KPA_angle)*head/mag)
    # --- Camber and Road Steer ---
    def camber(self, curr_KPA_angle):
        inclination = self.wheel_inclination(curr_KPA_angle)
        heading = self.wheel_heading(curr_KPA_angle)
        triple_product = np.cross(np.cross(heading, inclination), heading)
        mag = Vehicle.magnitude(triple_product)
        return np.sign(np.cross(inclination,heading)[2])*np.degrees(np.arccos(np.dot(triple_product, np.array([0,0,1]))/mag))
    def road_steer(self, curr_KPA_angle):
        return np.degrees(np.sign(curr_KPA_angle)*np.arccos(np.dot(self.wheel_heading(curr_KPA_angle),
                                                                   self.wheel_heading(0))/Vehicle.magnitude(self.wheel_heading(curr_KPA_angle))))
    # --- Steering Ratio ---
    def steering_ratio(self, curr_KPA_angle):
        """Steering Ratio is the Ratio of Steering Wheel Angle to the Road Steer Angle

        Args:
            curr_KPA_angle (float): Angle rotated by the Kingpin Axis 

        Returns:
            float: Steering Ratio
        """
        return -1/(self.road_steer(curr_KPA_angle)/self.rack_displacement(curr_KPA_angle)*2*np.pi*self.pinion/360)
    #  --- Regression Modeling for Inverse Functions, Kingpin Rotation Angle, Road Steering and Rack Displacement Correlated ---
    def regression_model(self):
        X = np.array([])
        y = np.array([])
        t = 0#-5self.step

        for i in range(0,50*self.conversionstep):
            t = t + self.step
            X = np.append(X,self.road_steer(t))
            y = np.append(y,t)
        t = 0
        for i in range(0,50*self.conversionstep):
            t = t - self.step
            X = np.append(X,self.road_steer(t))
            y = np.append(y,t)
        X = X.reshape(-1,1)
        
        poly_features = PolynomialFeatures(degree=25, include_bias=False)
        X_poly = poly_features.fit_transform(X)
        model = LinearRegression()
        model.fit(X_poly, y)
        X1 = np.array([])
        y1 = np.array([])
        t1 = 0 # -5self.step

        for i in range(0,50*self.conversionstep):
            t1 = t1 + self.step
            X1 = np.append(X1,self.rack_displacement(t1))
            y1 = np.append(y1,t1)
        t1 = 0
        for i in range(0,50*self.conversionstep):
            t1 = t1 - self.step
            X1 = np.append(X1,self.rack_displacement(t1))
            y1 = np.append(y1,t1)
        X1 = X1.reshape(-1,1)
        
        poly_features1 = PolynomialFeatures(degree=25, include_bias=False)
        X_poly1 = poly_features1.fit_transform(X1)
        model1 = LinearRegression()
        model1.fit(X1, y1)
        return model,poly_features,model1,poly_features1
    def KPA_rotation_angle(self, input_road_steer):
        input_road_steer = np.array([input_road_steer]).reshape(-1, 1)
        input_road_steer = self.model[1].fit_transform(input_road_steer)
        return (self.model[0].predict(input_road_steer))[0]
    def rack_vs_road_steer(self, input_road_steer):
        return self.rack_displacement(self.KPA_rotation_angle(input_road_steer))
    def KPA_rotation_angle_vs_rack(self, input_rack_stroke):
        input_rack_stroke = np.array([input_rack_stroke]).reshape(-1, 1)
        input_rack_stroke = self.model[3].fit_transform(input_rack_stroke)
        return (self.model[2].predict(input_rack_stroke))[0]
    def road_steer_vs_rack(self, input_rack_stroke):
        return self.road_steer(self.KPA_rotation_angle_vs_rack(input_rack_stroke))
    # --- Ackerman Calculations ---
    def ackerman(self, outer_angle):
        return np.degrees(np.arctan(self.wb/(self.wb/np.tan(np.radians(outer_angle))-self.tw)))
    def ackerman_percentage(self, inner, outer):
        return (inner - outer)/(self.ackerman(outer) - outer)*100
    def ackerman_vs_KPA(self, curr_KPA_angle):
        return self.ackerman_percentage(np.maximum(np.abs(self.road_steer(curr_KPA_angle)),
                                                   np.abs(self.road_steer(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle))))),
                                                   np.minimum(np.abs(self.road_steer(curr_KPA_angle)),
                                                              np.abs(self.road_steer(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle))))))
    # --- Tire Contact Patch positions: x_L, x_R, y_L, y_R ---
    def delta_T(self, curr_KPA_angle):
        return self.curr_T(curr_KPA_angle)-self.r_T
    def x_R(self, curr_KPA_angle):
        return self.delta_T(curr_KPA_angle)[0]
    def y_R(self, curr_KPA_angle):
        return self.delta_T(curr_KPA_angle)[1]
    def x_L(self, curr_KPA_angle):
        return self.delta_T(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle)))[0]
    def y_L(self, curr_KPA_angle):
        return -self.delta_T(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle)))[1]
    def z_R(self, curr_KPA_angle):
        return self.delta_z(curr_KPA_angle)
    def z_L(self, curr_KPA_angle):
        return self.delta_z(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle)))
    
    # --- Wheel Loads ----
    def F_L(self, curr_KPA_angle):
        """
        Curent KPA angle
        """
         # --- Wheel Loads Object ----
        return self.solve(curr_KPA_angle)[0]
    def F_R(self, curr_KPA_angle):
        """
        Curent KPA angle
        """
        return self.solve(curr_KPA_angle)[1]
    def R_L(self, curr_KPA_angle):
        """
        Curent KPA angle
        """
        return self.solve(curr_KPA_angle)[2]
    def R_R(self, curr_KPA_angle):
        """
        Curent KPA angle
        """
        return self.solve(curr_KPA_angle)[3]
    def FrontLoad(self, curr_KPA_angle):
        return self.F_L(curr_KPA_angle)+self.F_R(curr_KPA_angle)
    def RearLoad(self, curr_KPA_angle):
        return self.GVW - self.FrontLoad(curr_KPA_angle)
    def FLRR(self,curr_KPA_angle):
        return self.F_L(curr_KPA_angle)+self.R_R(curr_KPA_angle)
    def FRRL(self,curr_KPA_angle):
        return self.GVW - self.FLRR(curr_KPA_angle)
    # --- Set of Equations for Wheel Load Calculations ---
    def equation(self, x):
        zfl = x[0]
        zfr = x[1]
        zrl = x[2]
        zrr = x[3]
        Fl = self.Kf*zfl
        Fr = self.Kf*zfr
        Rl = self.Kr*zrl
        Rr = self.Kr*zrr
        

        FL = np.array([self.x_L(self.curr_KPA_angle), self.y_L(self.curr_KPA_angle), -self.z_L(self.curr_KPA_angle)-zfl])
        FR = np.array([self.x_R(self.curr_KPA_angle), self.tw+self.y_R(self.curr_KPA_angle), -self.z_R(self.curr_KPA_angle)-zfr])
        RL = np.array([self.a+self.b, 0, -zrl])
        RR = np.array([self.a+self.b, self.tw, -zrr])

        eq1 = Fl*(self.y_L(self.curr_KPA_angle)) + Rr*self.tw + Fr*(self.tw+self.y_R(self.curr_KPA_angle)) - self.GVW*self.tw/2
        eq2 = Fl*(self.a+self.b-self.x_L(self.curr_KPA_angle)) + Fr*(self.a+self.b-self.x_R(self.curr_KPA_angle)) - self.GVW*self.b
        eq3 = Fl + Fr + Rl + Rr - self.GVW
        eq4 = np.dot(np.cross(FR - FL,RL - FL), FR - RR)
        return [eq1,eq2,eq3,eq4]
    def solve(self, theta):
        self.curr_KPA_angle = theta
        F = self.GVW*self.b/(self.a+self.b)*0.5
        R = self.GVW*self.a/(self.a+self.b)*0.5
        [zfl,zfr,zrl,zrr] = (fsolve(self.equation,[F/self.Kf,F/self.Kf,R/self.Kr,R/self.Kr]))
        Fl = self.Kf*zfl
        Fr = self.Kf*zfr
        Rl = self.Kr*zrl
        Rr = self.Kr*zrr
        return Fl,Fr,Rl,Rr
    # --- Kingpin Moment Calulations ---
    def kpm_circular(self, theta):
        self.curr_KPA_angle = theta
        self.currKPA = (self.curr_A(theta)-self.curr_K(theta))/Vehicle.magnitude(self.r_A-self.r_K)
        t = theta
        normal = self.F_R(t)*np.array([0,0,1])/1000*g
        fric_dir = np.sign(t)*np.cross(normal,self.wheel_heading(t))
        friction = 0 # mu*F_R(t)*fric_dir/magnitude(fric_dir)/1000*g
        moment_arm = self.curr_T(t)-self.r_I
        total_force = friction+normal
        patch_radius = np.sqrt(self.F_R(self.curr_KPA_angle)*g/np.pi/self.tirep/6894.75729)
        temp = integrate.dblquad(self.tire_twisting_moment_circular, 0, 2*np.pi, 0, 1000*patch_radius)[0]/10**9
        return temp*np.sign(self.curr_KPA_angle) #np.dot(np.cross(moment_arm, total_force), KPA) + temp #+kpm_tp(curr_KPA_angle)
    def circular_contactpatch_element(self, r, phi):
        curr_point = self.curr_T(self.curr_KPA_angle) + np.array([r*np.cos(phi),r*np.sin(phi),0])
        return self.curr_tangent(curr_point)
    def tire_twisting_moment_circular(self, r,phi):
        distance = self.curr_T(self.curr_KPA_angle)+np.array([r*np.cos(phi),r*np.sin(phi),0])-self.r_I
        force = mu*self.tirep*6894.75729*r*self.circular_contactpatch_element(r,phi) + self.tirep*6894.75729*r*np.array([0,0,1]) #np.array([-np.sin(phi),np.cos(phi),0])
        return np.dot(np.cross(distance,force),self.currKPA)
    
    def static_kingpin_moment(self, curr_KPA_angle):
        return self.kpm_circular(curr_KPA_angle)
    # --- Steering Effort ---
    def tierod_force(self, curr_KPA_angle):
        self.currKPA = (self.curr_A(curr_KPA_angle)-self.curr_K(curr_KPA_angle))/Vehicle.magnitude(self.r_A-self.r_K)
        return self.static_kingpin_moment(curr_KPA_angle)*1000/np.dot(np.cross(self.steering_arm(curr_KPA_angle),
                                                                        self.tierod(curr_KPA_angle)/Vehicle.magnitude(self.tierod(curr_KPA_angle))),
                                                                        self.currKPA)*self.tierod(curr_KPA_angle)/Vehicle.magnitude(self.tierod(curr_KPA_angle))
    def rack_force(self, curr_KPA_angle):
        if curr_KPA_angle == 0:
            return 0
        else:
            return Vehicle.magnitude(self.tierod_force(curr_KPA_angle))**2/(np.dot(self.tierod_force(curr_KPA_angle),
                          np.array([0,1,0]))) - Vehicle.magnitude(self.tierod_force(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle))))**2/(np.dot(self.tierod_force(self.KPA_rotation_angle_vs_rack(-self.rack_displacement(curr_KPA_angle))),
                                                    np.array([0,1,0])))
    def static_steering_effort(self, curr_KPA_angle):
        if np.abs(curr_KPA_angle)<5:
            return np.abs(curr_KPA_angle/5*np.abs(self.rack_force(curr_KPA_angle)*self.pinion/1000))
        else:
            return np.abs(self.rack_force(curr_KPA_angle)*self.pinion/1000)
    # --- Plotting Functions ---
    def plotmycoordinates(self, func, title, legend, ylabel, xlabel):
        num = len(func)
        for j in range(num):
            X2 = np.array([])
            y2 = np.array([])
            t2 = self.KPA_rotation_angle_vs_rack(self.rack_stroke)
            max = self.KPA_rotation_angle_vs_rack(-self.rack_stroke)
            opop = int(max - t2)
            for i in range(0,opop*self.conversionstep):
                t2 = t2 + self.step
                X2 = np.append(X2,func[j](t2)[1])
                y2 = np.append(y2,func[j](t2)[0])
            X2 = X2.reshape(-1,1)
            plt.plot(X2,-y2)
            plt.scatter(func[j](0)[1], -func[j](0)[0])
            plt.scatter(func[j](self.KPA_rotation_angle_vs_rack(self.rack_stroke))[1], -func[j](self.KPA_rotation_angle_vs_rack(self.rack_stroke))[0])
            print('Initial ' + legend[j*3] + ' is ' + '('+str(round(func[j](0)[1],2)) + ', ' + str(round(func[j](0)[0],2)) +')')
            print('Extreme Values are (' + str(round(func[j](max)[1],2)) + 
                  ', '+str(round(func[j](max)[0],2)) + 
                  ') and (' + str(round((func[j](self.KPA_rotation_angle_vs_rack(self.rack_stroke)))[1],2)) + 
                  ', '+ str(round((func[j](self.KPA_rotation_angle_vs_rack(self.rack_stroke)))[0],2))+')')
        plt.legend(legend)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid()
    def plotmyfn(self, funcY, funcX, title, legend, ylabel, xlabel):
        num = len(funcY)
        for j in range(num):
            X2 = np.array([])
            y2 = np.array([])
            t2 = self.KPA_rotation_angle_vs_rack(self.rack_stroke)
            max = self.KPA_rotation_angle_vs_rack(-self.rack_stroke)
            opop = int(max - t2)
            for i in range(0,opop*self.conversionstep):
                t2 = t2 + self.step
                X2 = np.append(X2,funcX(t2))
                y2 = np.append(y2,funcY[j](t2))
            X2 = X2.reshape(-1,1)
            minim = round(np.min(y2),1)
            maxim = round(np.max(y2),1)
            plt.plot(X2,y2)
            plt.scatter( funcX(0),funcY[j](0))
            print('Initial ' + legend[j*2] + ' is ' + str(round(funcY[j](0),2)) )
            print('Extreme Values are ' + str(round(funcY[j](max),2)) + ' and ' + str(round((funcY[j](self.KPA_rotation_angle_vs_rack(self.rack_stroke))),2)))
            print('Range for ' + legend[j*2] + ' is ' + '['+ str(minim)+ ', '+str(maxim)+']')
            print('Average Absolute Value is ' + str(np.round(np.average(np.abs(y2)),2)))
        plt.legend(legend)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid()

### Inductive Development Flow

Define \( \zeta = [\alpha_L, \beta_L] \) as the domain for the right wheel Kingpin angle (in degrees), where:
- \( \alpha_L \) is the lock angle when steered right
- \( \beta_L \) is the lock angle when steered left

For example, for QUTE, \( \zeta = [-47.31^\circ, 37.28^\circ] \).

#### Case: \( \theta = 0^\circ \)

We know the initial position vectors:
\[ (\mathbf{r}_K)_0, (\mathbf{r}_A)_0, (\mathbf{r}_B)_0, (\mathbf{r}_O)_0, (\mathbf{r}_T)_0, (\mathbf{r}_W)_0, (\mathbf{r}_C)_0, (\mathbf{r}_{ICF})_0, (\mathbf{r}_{ICS})_0 \]

Here, \( (\mathbf{r}_M)_0 \) represents the initial position vector of point M.

#### For \( \theta = 0.1^\circ \):

1. **Rotation about KPA**:
   - Find \( (\mathbf{r}_B), (\mathbf{r}_O), (\mathbf{r}_T), (\mathbf{r}_W) \) as per the rotation about the Kingpin axis (KPA) given by \( (\mathbf{r}_A)_0 - (\mathbf{r}_K)_0 \).

2. **Virtual Bump/Droop**:
   - Find \( (\mathbf{r}_T), (\mathbf{r}_W) \) as per a virtual bump/droop, keeping the same Z-coordinate.

3. **Road-Steer Angle**:
   - Determine the road-steer angle as per \( \Delta (\mathbf{r}_{TW}) \).

4. **Virtual Bump/Droop for Fixed Ground**:
   - Find the virtual bump/droop \( (\delta z)_{\theta=0.1^\circ} \).

5. **Update Positions**:
   - Update \( (\mathbf{r}_O) \) after accounting for the virtual bump/droop \( (\delta z)_{\theta=0.1^\circ} \).
   - Find \( (\mathbf{r}_K), (\mathbf{r}_A) \) based on the rotation about the KPA given by \( (\mathbf{r}_{ICF})_0 - (\mathbf{r}_{ICS})_0 \) and the virtual bump/droop \( (\delta z)_{\theta=0.1^\circ} \).
   - Update \( (\mathbf{r}_B) \) considering the forced suspension path from \( \Delta (\mathbf{r}_K) \) (or \( \Delta (\mathbf{r}_A) \) if the steering arm is connected to point A).

6. **Tie-Rod Length Intersection**:
   - Find \( (\mathbf{r}_C) \) based on the intersection with \( (\mathbf{r}_C)_0 + \lambda \mathbf{j} \), where \( \lambda \) is a scalar.

7. **Rack Displacement**:
   - Calculate the rack displacement as per \( \Delta (\mathbf{r}_C) \).

#### For a General \( \theta = \theta_0 \in \zeta \):

Assume that after memoization, we know the position vectors:
\[ (\mathbf{r}_K)_{\theta_0}, (\mathbf{r}_A)_{\theta_0}, (\mathbf{r}_B)_{\theta_0}, (\mathbf{r}_O)_{\theta_0}, (\mathbf{r}_T)_{\theta_0}, (\mathbf{r}_W)_{\theta_0}, (\mathbf{r}_C)_{\theta_0}, (\mathbf{r}_{ICF})_{\theta_0}, (\mathbf{r}_{ICS})_{\theta_0}, (\delta z)_{\theta_0} \]

Where \( (\mathbf{r}_M)_{\theta_0} \) represents the position vector of point M at \( \theta = \theta_0 \).

For \( \theta = \theta_0 \pm 0.1^\circ \):

1. **Rotation about KPA**:
   - Find \( (\mathbf{r}_B), (\mathbf{r}_O), (\mathbf{r}_T), (\mathbf{r}_W) \) as per the rotation about the KPA given by \( (\mathbf{r}_A) - (\mathbf{r}_K) \) at \( \theta = \theta_0 \).

2. **Virtual Bump/Droop**:
   - Find \( (\mathbf{r}_T) \) keeping the same Z-coordinate as \( (\mathbf{r}_T)_{\theta_0} \).

3. **Total Road-Steer Angle**:
   - Determine the total road-steer angle as per \( \Delta (\mathbf{r}_{TW}) = (\mathbf{r}_{TW})_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_{TW})_0 \).

4. **Virtual Bump/Droop for Fixed Ground**:
   - Find \( (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} \).

5. **Update Positions**:
   - Update \( (\mathbf{r}_O) \) after accounting for the virtual bump/droop \( (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} \).
   - Find \( (\mathbf{r}_K), (\mathbf{r}_A) \) based on the rotation about the KPA given by \( (\mathbf{r}_{ICF}) - (\mathbf{r}_{ICS}) \) at \( \theta = \theta_0 \) and the virtual bump/droop \( (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} \).
   - Update \( (\mathbf{r}_B) \) considering the forced suspension path from \( \Delta (\mathbf{r}_K) = (\mathbf{r}_K)_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_K)_{\theta_0} \).

6. **Tie-Rod Length Intersection**:
   - Find \( (\mathbf{r}_C) \) based on the intersection with \( (\mathbf{r}_C)_{\theta_0} + \lambda \mathbf{j} \), where \( \lambda \) is a scalar.

7. **Total Rack Displacement**:
   - Calculate the total rack displacement as per \( \Delta (\mathbf{r}_C) = (\mathbf{r}_C)_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_C)_{\theta_0} \).

### Memoization

To facilitate this process, store the variables from \( 0 \) to \( \alpha_L \) in increments of \( 0.1^\circ \) and from \( 0 \) to \( \beta_L \) in decrements of \( 0.1^\circ \).

---

If there are any additional instructions or clarifications required, please let me know.Here's a professional rewrite of your text:

---

### Inductive Development Flow

Define $ \zeta = [\alpha_L, \beta_L] $ as the domain for the right wheel Kingpin angle (in degrees), where:
- $ \alpha_L $ is the lock angle when steered right
- $ \beta_L $ is the lock angle when steered left

For example, for QUTE, $ \zeta = [-47.31^\circ, 37.28^\circ] $.

#### Case: $ \theta = 0^\circ $

We know the initial position vectors:
$ (\mathbf{r}_K)_0, (\mathbf{r}_A)_0, (\mathbf{r}_B)_0, (\mathbf{r}_O)_0, (\mathbf{r}_T)_0, (\mathbf{r}_W)_0, (\mathbf{r}_C)_0, (\mathbf{r}_{ICF})_0, (\mathbf{r}_{ICS})_0 $

Here, $ (\mathbf{r}_M)_0 $ represents the initial position vector of point M.

#### For $ \theta = 0.1^\circ $:

1. **Rotation about KPA**:
   - Find $ (\mathbf{r}_B), (\mathbf{r}_O), (\mathbf{r}_T), (\mathbf{r}_W) $ as per the rotation about the Kingpin axis (KPA) given by $ (\mathbf{r}_A)_0 - (\mathbf{r}_K)_0 $.

2. **Virtual Bump/Droop**:
   - Find $ (\mathbf{r}_T), (\mathbf{r}_W) $ as per a virtual bump/droop, keeping the same Z-coordinate.

3. **Road-Steer Angle**:
   - Determine the road-steer angle as per $ \Delta (\mathbf{r}_{TW}) $.

4. **Virtual Bump/Droop for Fixed Ground**:
   - Find the virtual bump/droop $ (\delta z)_{\theta=0.1^\circ} $.

5. **Update Positions**:
   - Update $ (\mathbf{r}_O) $ after accounting for the virtual bump/droop $ (\delta z)_{\theta=0.1^\circ} $.
   - Find $ (\mathbf{r}_K), (\mathbf{r}_A) $ based on the rotation about the KPA given by $ (\mathbf{r}_{ICF})_0 - (\mathbf{r}_{ICS})_0 $ and the virtual bump/droop $ (\delta z)_{\theta=0.1^\circ} $.
   - Update $ (\mathbf{r}_B) $ considering the forced suspension path from $ \Delta (\mathbf{r}_K) $ (or $ \Delta (\mathbf{r}_A) $ if the steering arm is connected to point A).

6. **Tie-Rod Length Intersection**:
   - Find $ (\mathbf{r}_C) $ based on the intersection with $ (\mathbf{r}_C)_0 + \lambda \mathbf{j} $, where $ \lambda $ is a scalar.

7. **Rack Displacement**:
   - Calculate the rack displacement as per $ \Delta (\mathbf{r}_C) $.

#### For a General $ \theta = \theta_0 \in \zeta $:

Assume that after memoization, we know the position vectors:
$ (\mathbf{r}_K)_{\theta_0}, (\mathbf{r}_A)_{\theta_0}, (\mathbf{r}_B)_{\theta_0}, (\mathbf{r}_O)_{\theta_0}, (\mathbf{r}_T)_{\theta_0}, (\mathbf{r}_W)_{\theta_0}, (\mathbf{r}_C)_{\theta_0}, (\mathbf{r}_{ICF})_{\theta_0}, (\mathbf{r}_{ICS})_{\theta_0}, (\delta z)_{\theta_0} $

Where $ (\mathbf{r}_M)_{\theta_0} $ represents the position vector of point M at $ \theta = \theta_0 $.

For $ \theta = \theta_0 \pm 0.1^\circ $:

1. **Rotation about KPA**:
   - Find $ (\mathbf{r}_B), (\mathbf{r}_O), (\mathbf{r}_T), (\mathbf{r}_W) $ as per the rotation about the KPA given by $ (\mathbf{r}_A) - (\mathbf{r}_K) $ at $ \theta = \theta_0 $.

2. **Virtual Bump/Droop**:
   - Find $ (\mathbf{r}_T) $ keeping the same Z-coordinate as $ (\mathbf{r}_T)_{\theta_0} $.

3. **Total Road-Steer Angle**:
   - Determine the total road-steer angle as per $ \Delta (\mathbf{r}_{TW}) = (\mathbf{r}_{TW})_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_{TW})_0 $.

4. **Virtual Bump/Droop for Fixed Ground**:
   - Find $ (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} $.

5. **Update Positions**:
   - Update $ (\mathbf{r}_O) $ after accounting for the virtual bump/droop $ (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} $.
   - Find $ (\mathbf{r}_K), (\mathbf{r}_A) $ based on the rotation about the KPA given by $ (\mathbf{r}_{ICF}) - (\mathbf{r}_{ICS}) $ at $ \theta = \theta_0 $ and the virtual bump/droop $ (\delta z)_{\theta_0 \pm 0.1^\circ} - (\delta z)_{\theta_0} $.
   - Update $ (\mathbf{r}_B) $ considering the forced suspension path from $ \Delta (\mathbf{r}_K) = (\mathbf{r}_K)_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_K)_{\theta_0} $.

6. **Tie-Rod Length Intersection**:
   - Find $ (\mathbf{r}_C) $ based on the intersection with $ (\mathbf{r}_C)_{\theta_0} + \lambda \mathbf{j} $, where $ \lambda $ is a scalar.

7. **Total Rack Displacement**:
   - Calculate the total rack displacement as per $ \Delta (\mathbf{r}_C) = (\mathbf{r}_C)_{\theta_0 \pm 0.1^\circ} - (\mathbf{r}_C)_{\theta_0} $.

### Memoization

To facilitate this process, store the variables from $ 0 $ to $ \alpha_L $ in increments of $ 0.1^\circ $ and from $ 0 $ to $ \beta_L $ in decrements of $ 0.1^\circ $.